In [ ]:
import requests

In [ ]:
data_path = 'data'

Mounted at /drive


### Metric: Location Affordability Index : Version 3.0 (March 2019) – uses 2012-2016 American Community Survey

The Low Transportation Cost Index is based on estimates of transportation expenses for a family that meets the following description: 
-  hh6 = a 3-person single-parent family with income at 50% of the median income for renters for the region (i.e. CBSA). 

- **hh6_t_renters** (transportation costs as a percent of income for renters)
- **hh6_transit_trips_renters** (the number of public transit trips taken annually)

The estimates come from the Location Affordability Index (LAI). The data correspond to those for household type 6 (hh_type6_) as noted in the LAI data dictionary. More specifically, among this household type, we model transportation costs as a percent of income for renters (t_rent). Neighborhoods are defined as census tracts. The LAI data do not contain transportation cost information for Puerto Rico.

## Interpretation

**Metric: Transit trips index**

This metric reflects the number of public transit trips taken annually by a three-person single-parent
family with income at 50 percent of the AMI for renters. This number is percentile ranked nationally
into an index with values ranging from 0 to 100. Higher scores reflect better access to public
transportation.

**Metric: Low transportation cost index**

This index reflects local transportation costs as a share of renters’ incomes. It accounts for both transit
and cars. This index is based on estimates of transportation costs for a three-person, single-parent
family with income at 50 percent of the median income for renters for the region (i.e., a core-based
statistical area). Although other arrangements of family composition, income, and housing status are
possible in constructing this index, these characteristics were intended to more closely characterize a
lower-income household in the community. Values are inverted and percentile ranked nationally, with
values ranging from 0 to 100. The higher the value, the lower the cost of transportation in that
neighborhood.

 
Location Affordability Index : 
Version 3.0 (March 2019) – uses 2012-2016 American Community Survey data


### Dataset Description: 
The database generated by the foregoing methodology contains records for 72,241 U.S. Census tracts in the 50 states and the District of Columbia. The first set of fields (columns A-AL) contain data used to calibrate the model and then serve as inputs into the model to estimate household housing and transportation costs for the eight household profiles—including estimates for both owner and renter households—listed in Table 11 for each occupied Census tract. 

### API Documentation: 
https://developers.arcgis.com/rest/services-reference/enterprise/query-feature-service-layer-.htm

https://hudgis-hud.opendata.arcgis.com/datasets/location-affordability-index-v-3/api

There are about 440 variables so we need to look at which to examine
page 29 has variable explanation:
 https://files.hudexchange.info/resources/documents/Location-Affordability-Index-Version-3-Data-and-Methodology.pdf 

Variable of interest: 


In [ ]:
import requests
import time
import pandas as pd
def get_transportation_df(recreate=False):
  if recreate:
    final_features = []
    # trans_df = pd.DataFrame(columns=['GEOID','STATE','COUNTY','TRACT','CNTY_FIPS','STUSAB','hh6_transit_cost_renters','hh6_transit_trips_renters'])
    for i in range(1, 73):
      url = f"https://services.arcgis.com/VTyQ9soqVukalItT/arcgis/rest/services/Location_Affordability_Index_v3/FeatureServer/0/query?where=STATE={str(i).zfill(2)}&outFields=GEOID,STATE,COUNTY,TRACT,CNTY_FIPS,STUSAB,hh6_t_renters,hh6_transit_trips_renters&outSR=4326&f=json"
      state_response = requests.get(url)
      json_data = state_response.json()
      features = [row['attributes'] for row in json_data['features']]
      final_features.extend(features)
      time.sleep(3)
    trans_df = pd.DataFrame(final_features)
    trans_df.to_csv(f'{data_path}/interim/trans_index-7-20.csv')
  else:
    trans_df = pd.read_csv(f'{data_path}/interim/trans_index-7-20.csv', index_col=0)
  return trans_df

trans_df = get_transportation_df()

#map tract to city/place

In [ ]:
trans_df = pd.read_csv(f'{data_path}/interim/trans_index-7-20.csv', index_col=0,dtype={'place_fp': str})

In [ ]:
trans_df['hh6_t_renters'].describe()

count    37221.000000
mean        36.182307
std         10.532436
min          4.488886
25%         29.011102
50%         35.480153
75%         42.671902
max        102.934237
Name: hh6_t_renters, dtype: float64

In [ ]:

# drop PR all nans

trans_df = trans_df[(trans_df['STUSAB'].str.contains("PR")==False)]

In [ ]:
trans_df['transit_trips_index'] = trans_df['hh6_transit_trips_renters'].rank(pct = True) * 100

In [ ]:
trans_df['hh6_t_renters_inverse'] = 100/trans_df['hh6_t_renters']

In [ ]:
trans_df['transit_low_cost_index'] = trans_df['hh6_t_renters_inverse'].rank(pct = True) * 100

In [ ]:
trans_df

,GEOID,STATE,COUNTY,TRACT,CNTY_FIPS,STUSAB,hh6_t_renters,hh6_transit_trips_renters,transit_trips_index,hh6_t_renters_inverse,transit_low_cost_index
0,1089010800,1,89,10800,1089.0,AL,38.147100,75.562408,58.404118,2.621431,40.269340
1,1089010701,1,89,10701,1089.0,AL,37.595757,32.383889,34.589538,2.659875,42.215472
2,1059973000,1,59,973000,1059.0,AL,52.529053,0.000000,5.295414,1.903708,6.386753
3,1059973200,1,59,973200,1059.0,AL,57.049977,0.000000,5.295414,1.752849,3.282082
4,1089010401,1,89,10401,1089.0,AL,39.250520,66.926704,54.553519,2.547737,36.288372
...,...,...,...,...,...,...,...,...,...,...,...
37488,56007968100,56,7,968100,56007.0,WY,37.063094,79.688891,59.988710,2.698102,44.126660
37489,56007967600,56,7,967600,56007.0,WY,39.235329,112.861510,71.322778,2.548723,36.363636
37490,56007968000,56,7,968000,56007.0,WY,39.602153,29.117095,32.152841,2.525115,35.024999
37491,56021001000,56,21,1000,56021.0,WY,27.529776,96.546156,66.051556,3.632431,79.885490
